In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
frames = []
for csv in os.listdir('rowdata'):
    zipcode = csv[:5]
    df = pd.read_csv(os.path.join('rowdata', csv),  header = None, 
    names = ['name', 'price', 'mileage', 'exterior', 'interior', 'dealer_name', 'trans', 'engine', 'doors', 'photos', 
             'videos', 'moto_type'], quoting=3, error_bad_lines=False)
    df['zipcode'] = zipcode
    frames.append(df)
result = pd.concat(frames)

In [3]:
df = result
df = df.drop(['interior','dealer_name'], axis = 1)
df.dropna(axis = 'index', subset=['price', 'mileage', 'moto_type'], inplace = True)

In [4]:
df['certified'], df['year'], df['brand'], df['name'] = df.name.str.split(' ', 3).str
df['year'] = df['year'].map(lambda x: 2017 - int(x))
df = df[df['year'] >= 0]

df['photos'] = df.photos.str.split(' ', 1).str[0]
df['videos'] = df.videos.str.split(' ', 1).str[0]
df[['photos', 'videos']] = df[['photos', 'videos']].fillna(0)

df['doors'] = df.doors.str.split(' ', 1).str[0]
df['doors'] = df['doors'].map({'Four': 4, 'Two': 2, 'Three': 3, 'Five':5})
df['doors'] = df.groupby('name')['doors'].transform('max')

df['certified'] = df['certified'].map({'Used': 0, 'Certified': 1})

df.dropna(axis = 'index', subset=['exterior', 'trans', 'engine', 'doors'], inplace = True)

In [5]:
df.isnull().sum()

name         0
price        0
mileage      0
exterior     0
trans        0
engine       0
doors        0
photos       0
videos       0
moto_type    0
zipcode      0
certified    0
year         0
brand        0
dtype: int64

In [6]:
df.shape

(165587, 14)

In [7]:
df.head()

,name,price,mileage,exterior,trans,engine,doors,photos,videos,moto_type,zipcode,certified,year,brand
0,Maxima,11995.0,69406,Mystic Jade Metallic,Continuously Variable Automatic,6 Cylinder,4.0,39,1,sedan,02115,0,7,Nissan
1,335i xDrive Sedan,29995.0,25755,Jet Black,8 Speed Automatic,6 Cylinder Turbo,4.0,44,1,sedan,02115,0,3,BMW
2,528i xDrive Sedan,29995.0,13751,Dark Graphite Metallic,8 Speed Shiftable Automatic,4 Cylinder Turbo,4.0,50,1,sedan,02115,0,3,BMW
3,Q50 Premium,32990.0,13251,Moonlight White,7 Speed Shiftable Automatic,6 Cylinder,4.0,26,1,sedan,02115,1,2,INFINITI
4,A6 3.0T Prestige quattro,33995.0,27593,Ice Silver Metallic,8 Speed Shiftable Automatic,6 Cylinder Supercharg,4.0,57,1,sedan,02115,0,3,Audi


In [8]:
df.trans.value_counts().shape

(26,)

In [9]:
engine_type = df.engine.value_counts()[:19].index
df['engine'] = df['engine'].map(lambda x: x if x in engine_type else 'Others')
df.engine.value_counts().shape

(20,)

In [10]:
def find_color(x):
    for color in color_set:
        if color in x:
            return color
    return 'Others'

color_set = [_ for _ in df.exterior.value_counts().index if ' ' not in _][:19]
df['exterior'] = df['exterior'].map(find_color)
df.exterior.value_counts().shape

(20,)

In [11]:
df['name'] = df['name'].map(lambda x: x.split(' ', 1)[0])
df.name.value_counts().shape

(808,)

In [12]:
df = df[['year','brand', 'name','exterior', 'trans','engine', 'moto_type', 
         'zipcode','certified', 'mileage', 'photos','videos','doors','price']]
df.head()

,year,brand,name,exterior,trans,engine,moto_type,zipcode,certified,mileage,photos,videos,doors,price
0,7,Nissan,Maxima,Others,Continuously Variable Automatic,6 Cylinder,sedan,02115,0,69406,39,1,4.0,11995.0
1,3,BMW,335i,Black,8 Speed Automatic,6 Cylinder Turbo,sedan,02115,0,25755,44,1,4.0,29995.0
2,3,BMW,528i,Others,8 Speed Shiftable Automatic,4 Cylinder Turbo,sedan,02115,0,13751,50,1,4.0,29995.0
3,2,INFINITI,Q50,White,7 Speed Shiftable Automatic,6 Cylinder,sedan,02115,1,13251,26,1,4.0,32990.0
4,3,Audi,A6,Silver,8 Speed Shiftable Automatic,6 Cylinder Supercharg,sedan,02115,0,27593,57,1,4.0,33995.0


In [13]:
df.to_csv('660_dataset.csv')